In [ ]:
%%bash
# preamble script to check and install AMUSE components if necessary

# required packages for this tutorial:
PACKAGES="amuse-framework"
# skip in case a full development install is present
pip show amuse-devel && exit 0
for package in ${PACKAGES} 
do
  pip show ${package} || pip install ${package}
done

AMUSE tutorial on particle sets
====================


This tutorial introduces you the the most used data store in AMUSE, a *particle set*. AMUSE particle sets are a handy tool for storing data of stellar or planetary systems.

In [ ]:
from amuse.units import units, constants
from amuse.lab import Particles, Particle

A particle set store the properties of a system under study, for example:

In [ ]:
sun_and_earth = Particles(2)

of course at this point we have not specified anything about the system, lets change that:

In [ ]:
# declare the sun
sun = sun_and_earth[0]
sun.mass = 1 | units.MSun
sun.position = (0, 0, 0) | units.au
sun.velocity = (0, 0, 0) | units.kms
print(f"Sun={sun}")

In [ ]:
# now declare the Earth
def relative_orbital_velocity(mass, distance):
    return (constants.G * mass / distance).sqrt()

earth = sun_and_earth[1]
earth.mass = 1 | units.MEarth
earth.position = (1, 0, 0) | units.au

vorb = relative_orbital_velocity(
    sun_and_earth.total_mass(), 
    earth.position.length()
)
earth.velocity = (0, 1, 0) * vorb
print(f"Earth={earth}")

In [ ]:
# normally it is a good idea to study a system in its center of mass:
sun_and_earth.move_to_center()
print(sun_and_earth)

Now, let's give the particles a specific name (or other attribute)

In [ ]:
setattr(sun_and_earth, "name", "")
sun_and_earth.name = ["sun", "earth"]

How we have declared the particles and moved them to the center of mass.
We can also search for a specific particle.
For example, the one with the "earth" in the attribute "name".

In [ ]:
earth = sun_and_earth[sun_and_earth.name=="earth"][0] # note the index! do you understand the difference when removing it?
print(f"Earth={earth}")

We can add a moon in orbit around the earth

In [ ]:
moon = Particle()
moon.name = "moon"
moon.mass = 7.34767309e+22 | units.kg
moon.position = (384400, 0, 0) | units.km
vorb = relative_orbital_velocity(
    earth.mass + moon.mass, 
    moon.position.length()
)
moon.velocity = (0, 1, 0) * vorb
print(f"moon={moon}")

The Moon, however, is now somewhere inside the Sun with zero velocity - which is not good.
We will have to relocate the Moon to make it orbit around the Earth.
We do that by simply adding the positions and velocity of Earth to the Moon's.

In [ ]:
moon.position += earth.position
moon.velocity += earth.velocity

And we can add the moon to the Sun and Earth system

In [ ]:
sun_and_earth.add_particle(moon)

Note the use of the singular here, because we only add a single particle to the particle set sun_and_earth.
It is probably better to rename the sun_and_earth now.

In [ ]:
solar_system = sun_and_earth

It is important now to recenter the entire system, because by adding the moon we shifted the center of mass.

In [ ]:
solar_system.move_to_center()
print(f"Solar system:\n{solar_system}")

The key uniquely identifies a particle, but is otherwise never explicitly used:

In [ ]:
print(moon in solar_system)

We can now manipulate the planetary system, or query it.
for example by querying the masses.

In [ ]:
print("mass=", solar_system.mass.in_(units.MEarth))

This gives us a list of the masses of all objects, in units the the earth's mass. In fact, each of the particle's attributes is a simple numpy array: it can be assigned and manipulated as such.

Another way to acquire the same information could be done as follows:

In [ ]:
print(f"mass={solar_system.mass/solar_system[1].mass}")

Note that we get a reference to the attributes..lets make a copy of the solar system and play around with it:

In [ ]:
solar_system2=solar_system.copy()
x=solar_system2.x
print(x is solar_system2.x)
print(solar_system2.center_of_mass())
x[0]+=0.1| units.AU
print(solar_system2.center_of_mass())

But this is not necessarily true if the particle set is stored in a code, and can not be relied upon!

In some (hopefully rare) cases you may want to use the particle set or its attributes as simple numpy arrays, without the units.
This is easily achieved by stripping the unit from the array. This can be realied by explicitely querying the selected parameter with that specific unit.

In [ ]:
solar_system.position.value_in(units.parsec)

which, in this case, gives you a 2-dimensional *numpy.array* (a copy!) of the positions of star, planet and moon in units of a parsec.

Now, you may want to query the particle set solar_system.
For example by asking what attributes it has.
This can be done as follows:

In [ ]:
dir(solar_system)

Or get some general help on the underlying particle class

In [ ]:
help(solar_system)

You have performed some rudimentary operations on a particle set.
It is now time to experiment a little for yourself.


Assignments and questions:
---------------

### Assignment 1:
Add the planet Jupiter (see [Wikipedia](https://en.wikipedia.org/wiki/Jupiter)) to your small planetary system.

### Assignment 2:
Your planetary system is notoriously planar, and initialy the earth and moon are positioned along the Cartesian x-axis with the velocity vector in the Cartesian y-direction. 

Make the Sun-Earth-Moon system more realistic by introducing a small inclination to the Earth's and Moon's orbits and by giving them a random mean anomaly.

The [Orbital element module](https://github.com/amusecode/amuse/blob/main/src/amuse/ext/orbital_elements.py) of AMUSE could come in handy. 

### Assignment 3:
Calculate the total  gravitational binding energy of solarsystem.

Now displace the entire particle set by 100 parsec and give it a linear velocity of 100km/s in the z-direction.
Then calculate the binding energy of the system again.

Did the binding energy of the Solar system change by this translation?

### Question 1:
Particle sets have the attribute *get_binaries()*.
If you use this function to check the binaries in your system you will find that (without Jupiter) you have 3 binaries. Explain why the Sun is in a binary with the Moon. You may want to take a look at the [source code](https://github.com/amusecode/amuse/blob/main/src/amuse/datamodel/particle_attributes.py).

### Assignment 4:
Generate another particle set with a 2 solar-mass star and two planets of 10 and 100 Earth masses in circular orbits at 0.1 and 0.6 au.
Place this second planetary system at apocenter around your Solar system (true anomaly of 180 degrees) at a semimajor axis of 60 au with an eccentricity of 0.6.
Then move the entire system to the center of mass.

### Question 2:
Which of the orbits of the binary star with planets from *Assignment 4* has the highest binding energy?